In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from kerastuner.tuners import RandomSearch
import pandas as pd
import tensorflow as tf

<ipython-input-66-035e225ffc19>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
# Import our input dataset
companies_df = pd.read_csv("All.csv")
companies_df.head()

,sector,Name,symbol,code,year,Current ratio,Quick Ratio,Cash ratio,Operating cash flow ratio,Working capital ratio,...,EV / Free Cash Flow,EV / Invested Capital,EV / Revenue,P/E Ratio,Price/Book,Dividend Per Share,Altman Z-Score,Piotroski Score,Classification,ML_classification
0,Academic & Education,Global Education Communities,GEC,A&E 01,2013,0.63,0.61,0.20,-0.10,0.63,...,-4.0x,0.9x,0.9x,-5.9x,1.1x,0.0,-3.1,0,0,0
1,Academic & Education,Global Education Communities,GEC,A&E 01,2014,0.78,0.76,0.16,-0.25,0.78,...,-2.6x,0.7x,0.5x,-14.3x,1.0x,0.0,-0.9,6,0,0
2,Academic & Education,Global Education Communities,GEC,A&E 01,2015,0.27,0.26,0.05,0.08,0.27,...,16.6x,1.0x,1.9x,-18.7x,1.1x,0.0,-0.7,4,0,0
3,Academic & Education,Global Education Communities,GEC,A&E 01,2016,0.76,0.73,0.27,-0.04,0.76,...,84.6x,1.1x,2.4x,6.4x,1.5x,0.0,2.9,4,1,0
4,Academic & Education,Global Education Communities,GEC,A&E 01,2017,0.32,0.31,0.12,0.18,0.32,...,28.4x,1.2x,3.4x,30.6x,1.8x,0.0,1.9,4,1,0


In [ ]:
# Replacing x suffix from the DataFrame

columns_with_x = ['EV/EBITDA', 'EV / EBIT', 'EV / Free Cash Flow',
                    'EV / Invested Capital', 'EV / Revenue', 'P/E Ratio', 'Price/Book']

# Function to remove 'x' and convert to numeric
def remove_x_convert_numeric(value):
    if isinstance(value, str):
        return pd.to_numeric(value.replace('x', ''), errors='coerce')
    return value

# Apply the function to the specified columns
for column in columns_with_x:
    companies_df[column] = companies_df[column].apply(remove_x_convert_numeric)


In [ ]:
companies_df.head()

,sector,Name,symbol,code,year,Current ratio,Quick Ratio,Cash ratio,Operating cash flow ratio,Working capital ratio,...,EV / Free Cash Flow,EV / Invested Capital,EV / Revenue,P/E Ratio,Price/Book,Dividend Per Share,Altman Z-Score,Piotroski Score,Classification,ML_classification
0,Academic & Education,Global Education Communities,GEC,A&E 01,2013,0.63,0.61,0.20,-0.10,0.63,...,-4.0,0.9,0.9,-5.9,1.1,0.0,-3.1,0,0,0
1,Academic & Education,Global Education Communities,GEC,A&E 01,2014,0.78,0.76,0.16,-0.25,0.78,...,-2.6,0.7,0.5,-14.3,1.0,0.0,-0.9,6,0,0
2,Academic & Education,Global Education Communities,GEC,A&E 01,2015,0.27,0.26,0.05,0.08,0.27,...,16.6,1.0,1.9,-18.7,1.1,0.0,-0.7,4,0,0
3,Academic & Education,Global Education Communities,GEC,A&E 01,2016,0.76,0.73,0.27,-0.04,0.76,...,84.6,1.1,2.4,6.4,1.5,0.0,2.9,4,1,0
4,Academic & Education,Global Education Communities,GEC,A&E 01,2017,0.32,0.31,0.12,0.18,0.32,...,28.4,1.2,3.4,30.6,1.8,0.0,1.9,4,1,0


In [ ]:
# Define the Columns to Convert to float
columns_to_convert = [
    'Interest coverage ratio',
    'The Long-Term Debt Coverage Ratio',
    'Total debt to EBITDA ratio',
    'Inventory turnover ratio',
    'Receivables turnover ratio',
    'Working capital turnover ratio',
    'EBITDA'
]

def convert_to_float(df, columns):
    for column in columns:
        # Remove commas and convert to float
        df[column] = pd.to_numeric(df[column].str.replace(',', ''), errors='coerce')

# Apply the conversion function to the DataFrame
convert_to_float(companies_df, columns_to_convert)


In [ ]:
# Drop columns from the DataFrame
companies_df.drop(columns=['sector', 'symbol', 'code'], inplace=True)

In [ ]:
# Convert categorical data to numeric using one-hot encoding
dummies_companies_df = pd.get_dummies(companies_df)

In [ ]:
# render_username = 'forecating_companies_future_user'
# render_password = 'qRvkrYzSYuo6TvDWAh8SMcQokhT5pYyb'
# render_host = 'dpg-cog4tdmv3ddc73e67q00-a.ohio-postgres.render.com'
# database = 'forecating_companies_future'
# engine = create_engine(f"postgresql+psycopg2://{render_username}:{render_password}@{render_host}:5432/{database}")

NameError: name 'create_engine' is not defined

In [ ]:
# df.to_sql('Company_Data', engine)

In [ ]:
# Splitting the data into features and target
y = dummies_companies_df['ML_classification'].values
X = dummies_companies_df.drop(columns='ML_classification').values

In [ ]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#Define the Model with Keras Tuner
def create_model(hp):
    model = tf.keras.models.Sequential()
    activation = hp.Choice('activation', ['sigmoid', 'relu', 'tanh'])

    # First layer
    model.add(tf.keras.layers.Dense(
        units=hp.Int('first_units', min_value=10, max_value=100, step=10),
        activation=activation, input_dim=X_train_scaled.shape[1]))

    # Additional hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):  # You can adjust the range of layers
        model.add(tf.keras.layers.Dense(
            units=hp.Int(f'units_layer_{i}', min_value=10, max_value=100, step=10),
            activation=activation))

    # Output layer
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
!pip install keras-tuner

In [ ]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 06s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 03m 43s


In [ ]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 50, 'num_layers': 3, 'units_layer_0': 50, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0, 'units_layer_1': 10, 'units_layer_2': 10}
{'activation': 'relu', 'first_units': 90, 'num_layers': 3, 'units_layer_0': 60, 'units_layer_1': 100, 'units_layer_2': 50, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 50, 'num_layers': 1, 'units_layer_0': 30, 'units_layer_1': 30, 'units_layer_2': 60, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}


In [ ]:
# Evaluate the top 3 models against the test dataset
top_models = tuner.get_best_models(3)
for i, model in enumerate(top_models, start=1):
    model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
    print(f"Model {i}: Loss: {model_loss}, Accuracy: {model_accuracy}")

14/14 - 0s - loss: nan - accuracy: 1.0000 - 247ms/epoch - 18ms/step
Model 1: Loss: nan, Accuracy: 1.0
14/14 - 0s - loss: nan - accuracy: 1.0000 - 277ms/epoch - 20ms/step
Model 2: Loss: nan, Accuracy: 1.0
14/14 - 0s - loss: nan - accuracy: 1.0000 - 229ms/epoch - 16ms/step
Model 3: Loss: nan, Accuracy: 1.0


In [ ]:
# Creating the model with the specified hyperparameters
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(50, activation='relu', input_dim=X_train_scaled.shape[1]),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2)

# Evaluate the model using the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

Epoch 1/10
34/34 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.9944 - val_loss: nan - val_accuracy: 1.0000
Epoch 2/10
34/34 [==============================] - 0s 3ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 3/10
34/34 [==============================] - 0s 4ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 4/10
34/34 [==============================] - 0s 3ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 5/10
34/34 [==============================] - 0s 3ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 6/10
34/34 [==============================] - 0s 4ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 7/10
34/34 [==============================] - 0s 4ms/step - loss: nan - accuracy: 1.0000 - val_loss: nan - val_accuracy: 1.0000
Epoch 8/10
34/34 [==============================] - 0s 4ms/ste

In [ ]:
# Saving the model
model.save("Forecasting_bankrupt_companies.h5")